## Guilty Inventory Report (Pandas Version)

1.  Put csv data files as below.  Each csv files must have datetime format for the dates.

```
inventory/
 ├─ pandas_ginv.ipynb
 ├─ csv_data/         -- aging.csv, business_unit.csv, rate.csv
 │   ├─inv_data       -- invoice csv files
 │   ├─purchase_data  -- purchase csv files
 │   └─sales_data     -- sales csv files
 └─ reports
 
```
2.  From Jupyter, run the whole notebook.
 
3.  Reports are saved in reports folder.
 

In [1]:
import glob
import logging
import os
from datetime import date, timedelta

import pandas as pd
from dateutil.relativedelta import relativedelta

logging.basicConfig(
    level=logging.INFO, format=" %(asctime)s -  %(levelname)s -  %(message)s"
)
# logging.disable(logging.CRITICAL)

In [2]:
def main():
#     read_inv()
#     read_sales()
#     read_purchase()
#     read_bu()
#     read_rate()
#     read_aging()
#     latest_ps()
#     new_age()
    bs_report_act_age("2020/3", "2020/8")
    bs_report_new_age("2020/3", "2020/8")
#     digest_act_age()
    admin_report_act_age("2020/8")
#     digest_new_age()
    admin_report_new_age("2020/8")
    

In [3]:
def read_inv():
    """Reads csv_data/inv_data/*.csv
    
    Inventory aging preparation from csv data file.
    Copy the original excel sheet to a new sheet & paste value and save as csv.
    Add columun "inv_month" with date format like 2019/9/1. (day must be 1st day)    
    """

    list = []

    for files in glob.glob("csv_data/inv_data/*.csv"):
        useCols = [
            "CatNo",
            "Material",
            "Fecha",
            "Suma de Total Qty",
            "Suma de Total $",
            "inv_month",
        ]
        csv = pd.read_csv(files, usecols=useCols, parse_dates=[5])
        list.append(csv)

    df = pd.concat(list)
    df = df.rename(
        columns={
            "CatNo": "cat",
            "Material": "model",
            "Fecha": "age",
            "Suma de Total Qty": "qty",
            "Suma de Total $": "amt",
            "inv_month": "month",
        }
    )

    df = df.reindex(columns=["month", "model", "cat", "qty", "amt", "age"])

    # Replace the aging date by integer
    agedict = {
        "30D": 30,
        "60D": 60,
        "90D": 90,
        "180D": 180,
        "270D": 270,
        "360D": 360,
        "MAS": 999,
    }
    df = df.replace(agedict)

    # Make sure the month to have 1st date of the month.
    df["month"] = df["month"].apply(lambda x: x.replace(day=1))

    # Grouping by month and model.
    grouped = df.groupby(["month", "model", "cat", "age"], as_index=False)
    df = grouped.sum()
    #     df = df[df['qty'] > 0]
    
    logging.info('read_inv : Successfully finished.')
    return df


logging.debug("read_inv() : return: \n" + str(read_inv()))

 2020-09-30 17:23:01,338 -  INFO -  read_inv : Successfully finished.


In [4]:
def read_sales():
    """Reads csv_data/sales_data/*.csv
    
    Sales record preparation from csv file.
    Copy the original excel sheet to a new sheet & paste value and save as csv.
    Billing doc. date must be formatted to date (ex: 2020/8/2)   
    """

    list = []

    for targetFile in glob.glob("csv_data/sales_data/*.csv"):
        useCols = ["Billing doc/ date", "Material", "Net Qty", "Net $"]
        csv = pd.read_csv(targetFile, usecols=useCols, parse_dates=[0])
        list.append(csv)

    df = pd.concat(list)
    df = df.rename(
        columns={
            "Billing doc/ date": "month",
            "Material": "model",
            "Net Qty": "qty",
            "Net $": "amt",
        }
    )

    # Modify the sales month to have 1st date of the month
    df["month"] = df["month"].apply(lambda x: x.replace(day=1))
    grouped = df.groupby(["month", "model"], as_index=False)
    df = grouped.sum()
    df = df[df["qty"] > 0]
    logging.info("read_sales() : Successfully finished.")

    return df


logging.debug("read_sales: return: \n" + str(read_sales()))

 2020-09-30 17:23:01,776 -  INFO -  read_sales() : Successfully finished.


In [5]:
def read_purchase():
    """Reads csv_data/purchase_data/*.csv
    
    Purchase preparation from csv file.
    Copy the original excel sheet to a new sheet & paste value and save as csv.    
    """

    list = []

    for files in glob.glob("csv_data/purchase_data/*.csv"):
        useCols = ["Material", "Posting date", "Net Qty", "Net - $"]
        csv = pd.read_csv(files, header=0, usecols=useCols, parse_dates=[1])
        list.append(csv)

    df = pd.concat(list)
    df = df.rename(
        columns={
            "Material": "model",
            "Net Qty": "qty",
            "Net - $": "amt",
            "Posting date": "month",
        }
    )

    df = df.reindex(columns=["month", "model", "qty", "amt"])

    # Modify the month to have 1st date of the month
    df["month"] = df["month"].apply(lambda x: x.replace(day=1))

    # Grouping by in_month and inv_model
    grouped = df.groupby(["month", "model"], as_index=False)
    df = grouped.sum()
    df = df[df["qty"] > 0]
    logging.info('read_purchase() : Successfully finished.')

    return df


logging.debug("read_purchase: return: \n" + str(read_purchase()))

 2020-09-30 17:23:01,832 -  INFO -  read_purchase() : Successfully finished.


In [6]:
def read_bu():
    """Reads csv_data/business_unit.csv
    
    business_unit preparation from csv file.
    """

    df = pd.read_csv("csv_data/business_unit.csv")
    df = df.rename(
        columns={
            "prod_category": "cat",
            "business_unit": "bu",
            "bu_description": "bu_name",
        }
    )
    logging.info('read_bu() : Successfully finished.')

    return df


logging.debug("read_bu: return: \n" + str(read_bu()))

 2020-09-30 17:23:01,849 -  INFO -  read_bu() : Successfully finished.


In [7]:
def read_rate():
    """Reads csv_data/rate.csv
    
    Rate table preparation from csv file.
    """ 

    df = pd.read_csv("csv_data/rate.csv", parse_dates=[0])
    df = df.rename(columns={"rate_month": "month"})

    # Modify the inv_month to have 1st date of the month, just in case.
    df["month"] = df["month"].apply(lambda x: x.replace(day=1))
    logging.info('read_rate() : Successfully finished.')

    return df


logging.debug("read_rate: return: \n" + str(read_rate()))

 2020-09-30 17:23:01,873 -  INFO -  read_rate() : Successfully finished.


In [8]:
def read_aging():
    """Reads csv_data/agin.csv
    
    Inventory aging convertion table preparation from csv file.
    The table is used to get new age of the inventory.
    """ 


    df = pd.read_csv("csv_data/aging.csv", index_col=0)
    df = df.rename(
        columns={
            "age_n0": 0,
            "age_n1": 1,
            "age_n2": 2,
            "age_n3": 3,
            "age_n4": 4,
            "age_n5": 5,
            "age_n6": 6,
            "age_n7": 7,
            "age_n8": 8,
            "age_n9": 9,
            "age_n10": 10,
            "age_n11": 11,
            "age_n12": 12,
            "age_n13": 13,
        }
    )

    df = df.stack()
    df = df.reset_index()
    df = df.rename(columns={"age_actual": "age", "level_1": "month_diff", 0: "new_age"})
    logging.info('read_aging() : Successfully finished.')

    return df


logging.debug("read_aging: return: \n" + str(read_aging()))

 2020-09-30 17:23:01,900 -  INFO -  read_aging() : Successfully finished.


In [9]:
def latest_ps():
    """Get df which has the movements of sales or purchase.
    
    This is used to get the month difference between inventory month and the latest
    p or s movement to get new age according to aging conversion table.
    """

    df1 = read_sales()
    df1 = df1[["month", "model"]]
    df2 = read_purchase()
    df2 = df2[["month", "model"]]
    df = df1.append(df2)

    # Make a dataframe which has the latest p or s movement by month and model.
    df = df.drop_duplicates()
    df = df.sort_values(["month", "model"], ascending=[False, True])
    logging.info("latest_ps() : Successfully finished.")

    return df


logging.debug("return : \n" + str(latest_ps()))

 2020-09-30 17:23:02,359 -  INFO -  read_sales() : Successfully finished.
 2020-09-30 17:23:02,403 -  INFO -  read_purchase() : Successfully finished.
 2020-09-30 17:23:02,414 -  INFO -  latest_ps() : Successfully finished.


In [10]:
def new_age():
    """Get df of inventory with the new age.""" 

    try:
        df1 = read_inv()
        df1 = df1.drop(["cat", "qty", "amt"], axis=1)
        df2 = latest_ps()
        df = pd.merge(df1, df2, on="model", how="left", suffixes=["_inv", "_ps"])
        df = df.loc[df["month_inv"] >= df["month_ps"]]

        # Make a dataframe which has the latest p or s movement by model.
        grouped = df.groupby(["month_inv", "model", "age"])
        df = df.loc[grouped["month_ps"].idxmax(), :]

        # Add month_diff column
        df["month_diff"] = (df["month_inv"].dt.year - df["month_ps"].dt.year) * 12 + (
            df["month_inv"].dt.month - df["month_ps"].dt.month
        )

        # Make inv list recovering the dropped item with month diff = 13.
        df = df.rename(columns={"month_inv": "month"})
        df = pd.merge(df1, df, on=["month", "model", "age"], how="left")
        df = df.fillna({"month_diff": 13})

        # Convert month_diff to new_age
        df3 = read_aging()
        df = pd.merge(df, df3, on=["age", "month_diff"], how="left")

    except Exception as e:
        logging.exception('Raise Exception: %s', e)

    else:
        logging.info('new_age() : Successfully finished.')
        return df


logging.debug("return : \n" + str(new_age()))

 2020-09-30 17:23:02,602 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:03,037 -  INFO -  read_sales() : Successfully finished.
 2020-09-30 17:23:03,075 -  INFO -  read_purchase() : Successfully finished.
 2020-09-30 17:23:03,087 -  INFO -  latest_ps() : Successfully finished.
 2020-09-30 17:23:04,039 -  INFO -  read_aging() : Successfully finished.
 2020-09-30 17:23:04,048 -  INFO -  new_age() : Successfully finished.


In [11]:
def bs_report_act_age(start, end):
    """Make csv report file for bs with actual age.
    
    Args:
        start (str): start of inventory month
        end (str): end of inventory month
    """ 

    try:
        df1 = read_inv()
        df1 = df1.query("@start <= month <= @end")
        df2 = read_bu()
        df = pd.merge(df1, df2, on="cat", how="left")
        df = df.drop(["cat", "bu_name"], axis=1)
        df = df.query('bu in ["D1", "D2"]')
        df = df.reindex(columns=["month", "bu", "model", "age", "qty", "amt"])
        df.to_csv("reports/bs_actual.csv", index=False)

    except Exception as e:
        logging.exception('Raise Exception: %s', e)

    else:
        logging.info("bs_report_act_age(" + start + ", " + end + "): Successfully finished.")


logging.debug("return : \n" + str(bs_report_act_age("2020/3", "2020/8")))

 2020-09-30 17:23:04,256 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:04,267 -  INFO -  read_bu() : Successfully finished.
 2020-09-30 17:23:04,338 -  INFO -  bs_report_act_age(2020/3, 2020/8): Successfully finished.


In [12]:
def bs_report_new_age(start, end):
    """Make csv report file for bs with new age.
    
    Args:
        start (str): start of inventory month
        end (str): end of inventory month
    """ 

    try:
        df1 = read_inv()
        df1.to_csv("inv.csv")
        df2 = new_age()
        df1 = pd.merge(df1, df2, on=["month", "model", "age"], how="left")
        df1 = df1.drop(["age", "month_ps", "month_diff"], axis=1)
        df1 = df1.query("@start <= month <= @end")
        df2 = read_bu()
        df = pd.merge(df1, df2, on="cat", how="left")
        df = df.drop(["cat", "bu_name"], axis=1)
        df = df.query('bu in ["D1", "D2"]')
        df = df.reindex(columns=["month", "bu", "model", "new_age", "qty", "amt"])

        # Grouping by month and model.
        grouped = df.groupby(["month", "bu", "model", "new_age"], as_index=False)
        df = grouped.sum()
        df.to_csv("reports/bs_new_age.csv", index=False)

    except Exception as e:
        logging.exception('Raise Exception: %s', e)
        
    else:
        logging.info("bs_report_new_age(" + start + ", " + end + "): Successfully finished.")
        return df        


logging.debug("return : \n" + str(bs_report_new_age("2020/3", "2020/8")))

 2020-09-30 17:23:04,553 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:04,828 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:05,299 -  INFO -  read_sales() : Successfully finished.
 2020-09-30 17:23:05,340 -  INFO -  read_purchase() : Successfully finished.
 2020-09-30 17:23:05,351 -  INFO -  latest_ps() : Successfully finished.
 2020-09-30 17:23:06,315 -  INFO -  read_aging() : Successfully finished.
 2020-09-30 17:23:06,323 -  INFO -  new_age() : Successfully finished.
 2020-09-30 17:23:06,343 -  INFO -  read_bu() : Successfully finished.
 2020-09-30 17:23:06,394 -  INFO -  bs_report_new_age(2020/3, 2020/8): Successfully finished.


In [13]:
def digest_act_age():
    """Get digest models for admin report with actual age."""

    try:
        df_inv = read_inv()
        df_rate = read_rate()
        df = pd.merge(df_inv, df_rate, on="month", how="left")
        df = df[df["age"] > 90]
        df["yen"] = df["amt"] * df["rate_yen"]
        df = df.drop(["cat", "qty", "amt", "age", "rate_yen"], axis=1)
        grouped = df.groupby(["month", "model"], as_index=False)
        df = grouped.sum()

        df["digest_model"] = df["model"]
        df.loc[df["yen"] < 1000000, "digest_model"] = "Others"
        df = df.drop(["yen"], axis=1)

    except Exception as e:
        logging.exception("Raise Exception: %s", e)

    else:
        logging.info("digest_act_age(): Successfully finished.")
        return df


logging.debug("return : \n" + str(digest_act_age()))

 2020-09-30 17:23:06,569 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:06,578 -  INFO -  read_rate() : Successfully finished.
 2020-09-30 17:23:06,598 -  INFO -  digest_act_age(): Successfully finished.


In [14]:
def admin_report_act_age(targetMonth):
    """Make csv report file for admin with actual age.
    
    Args:
        targetMonth (str): inventory month to report.
    """ 

    df_inv = read_inv()

    # Only guilty inventory
    df_inv = df_inv[(df_inv["age"] > 90)]

    # Add BU info
    df_bu = read_bu()
    df_inv = pd.merge(df_inv, df_bu, on="cat", how="left")
    df_inv = df_inv.drop(["cat", "age"], axis=1)
    grouped = df_inv.groupby(["month", "model", "bu"], as_index=False)
    df_inv = grouped.sum()

    # Add digest model and set to df_report
    df_digest = digest_act_age()
    df_inv = pd.merge(df_inv, df_digest, on=["month", "model"], how="left")
    df_report = df_inv.copy()

    # Add N, N-1 month column to join sales & inv
    df_report["n-0"] = df_report["month"]
    df_report["n-1"] = df_report["month"].map(lambda x: x - relativedelta(months=1))

    # Add N, N-1 sales data to df_report
    df_sales = read_sales()
    df_report = pd.merge(
        df_report,
        df_sales,
        left_on=["n-0", "model"],
        right_on=["month", "model"],
        how="left",
        suffixes=["", "_s0"],
    )

    df_report = pd.merge(
        df_report,
        df_sales,
        left_on=["n-1", "model"],
        right_on=["month", "model"],
        how="left",
        suffixes=["", "_s1"],
    )

    # Add N-1 inv data to df_report
    df_report = pd.merge(
        df_report,
        df_inv,
        left_on=["n-1", "model"],
        right_on=["month", "model"],
        how="left",
        suffixes=["", "_i1"],
    )

    # Group by digest model
    df_report = df_report.drop(
        [
            "model",
            "n-0",
            "n-1",
            "month_s0",
            "month_s1",
            "month_i1",
            "digest_model_i1",
            "bu_i1",
            "amt_i1",
        ],
        axis=1,
    )

    grouped = df_report.groupby(["month", "bu", "digest_model"], as_index=False)
    df_report = grouped.sum()

    df_report["amt"] = df_report["amt"].map(lambda x: x / 1000)
    df_report["amt_s1"] = df_report["amt_s1"].map(lambda x: x / 1000)
    df_report["amt_s0"] = df_report["amt_s0"].map(lambda x: x / 1000)

    df_report = df_report.reindex(
        columns=[
            "month",
            "bu",
            "digest_model",
            "qty_i1",
            "qty",
            "amt",
            "qty_s1",
            "amt_s1",
            "qty_s0",
            "amt_s0",
        ]
    )

    df_report = df_report.rename(
        columns={
            "digest_model": "model",
            "qty_i1": "last inv_q",
            "qty": "act_inv_q",
            "amt": "act_inv_kmxt",
            "qty_s1": "last_s_q",
            "amt_s1": "last_s_kmx",
            "qty_s0": "act_s_q",
            "amt_s0": "act_s_kmx",
        }
    )

    df_report = df_report.query("month == @targetMonth")
    
    try:
        df_report.to_csv("reports/admin_report_act_age.csv", index=False)
    except PermissionError as e:
        logging.exception("Raise Exception: %s", e)
    else:
        logging.info("admin_report_act_age(" + targetMonth + "): Successfully finished.")



logging.debug("return : \n" + str(admin_report_act_age("2020/8")))

 2020-09-30 17:23:06,792 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:06,798 -  INFO -  read_bu() : Successfully finished.
 2020-09-30 17:23:07,015 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:07,023 -  INFO -  read_rate() : Successfully finished.
 2020-09-30 17:23:07,044 -  INFO -  digest_act_age(): Successfully finished.
 2020-09-30 17:23:07,617 -  INFO -  read_sales() : Successfully finished.
 2020-09-30 17:23:07,670 -  INFO -  admin_report_act_age(2020/8): Successfully finished.


In [15]:
def digest_new_age():
    """Get digest models for admin report with new age.""" 

    df_inv = read_inv()
    df_newage = new_age()
    df_inv = pd.merge(df_inv, df_newage, on=['month', 'model', 'age'], how='left')
    df_inv = df_inv.drop(["age", "month_ps", "month_diff"], axis=1)
    df_inv = df_inv.rename(columns={"new_age": "age"})
    grouped = df_inv.groupby(["month", "model", "cat", "age"], as_index=False)
    df_inv = grouped.sum()

    df_rate = read_rate()
    df_digest = pd.merge(df_inv, df_rate, on="month", how="left")
    df_digest = df_digest[df_digest["age"] > 90]
    df_digest["yen"] = df_digest["amt"] * df_digest["rate_yen"]
    df_digest = df_digest.drop(["cat", "qty", "amt", "age", "rate_yen"], axis=1)
    grouped = df_digest.groupby(["month", "model"], as_index=False)
    df_digest = grouped.sum()
    df_digest["digest_model"] = df_digest["model"]
    df_digest.loc[df_digest["yen"] < 1000000, "digest_model"] = "Others"
    df_digest = df_digest.drop(["yen"], axis=1)
    
    logging.info("digest_new_age: Successfully finished.")
    return df_digest


logging.debug("return : \n" + str(digest_new_age()))

 2020-09-30 17:23:07,876 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:08,033 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:08,477 -  INFO -  read_sales() : Successfully finished.
 2020-09-30 17:23:08,517 -  INFO -  read_purchase() : Successfully finished.
 2020-09-30 17:23:08,526 -  INFO -  latest_ps() : Successfully finished.
 2020-09-30 17:23:09,461 -  INFO -  read_aging() : Successfully finished.
 2020-09-30 17:23:09,470 -  INFO -  new_age() : Successfully finished.
 2020-09-30 17:23:09,501 -  INFO -  read_rate() : Successfully finished.
 2020-09-30 17:23:09,522 -  INFO -  digest_new_age: Successfully finished.


In [16]:
def admin_report_new_age(targetMonth):
    """Make csv report file for admin with new age.
    
    Args:
        targetMonth (str): inventory month to report.
    """ 

    df_inv = read_inv()
    df_newage = new_age()
    df_inv = pd.merge(df_inv, df_newage, on=['month', 'model', 'age'], how='left')
    df_inv = df_inv.drop(["age", "month_ps", "month_diff"], axis=1)
    df_inv = df_inv.rename(columns={"new_age": "age"})
    grouped = df_inv.groupby(["month", "model", "cat", "age"], as_index=False)
    df_inv = grouped.sum()    

    # Only guilty inventory
    df_inv = df_inv[(df_inv["age"] > 90)]

    # Add BU info
    df_bu = read_bu()
    df_inv = pd.merge(df_inv, df_bu, on="cat", how="left")
    df_inv = df_inv.drop(["cat", "age"], axis=1)
    grouped = df_inv.groupby(["month", "model", "bu"], as_index=False)
    df_inv = grouped.sum()

    # Add digest model and set to df_report
    df_digest = digest_new_age()
    df_inv = pd.merge(df_inv, df_digest, on=["month", "model"], how="left")
    df_report = df_inv.copy()

    # Add N, N-1 month column to join sales & inv
    df_report["n-0"] = df_report["month"]
    df_report["n-1"] = df_report["month"].map(lambda x: x - relativedelta(months=1))

    # Add N, N-1 sales data to df_report
    df_sales = read_sales()
    df_report = pd.merge(
        df_report,
        df_sales,
        left_on=["n-0", "model"],
        right_on=["month", "model"],
        how="left",
        suffixes=["", "_s0"],
    )

    df_report = pd.merge(
        df_report,
        df_sales,
        left_on=["n-1", "model"],
        right_on=["month", "model"],
        how="left",
        suffixes=["", "_s1"],
    )

    # Add N-1 inv data to df_report
    df_report = pd.merge(
        df_report,
        df_inv,
        left_on=["n-1", "model"],
        right_on=["month", "model"],
        how="left",
        suffixes=["", "_i1"],
    )

    # Group by digest model
    df_report = df_report.drop(
        [
            "model",
            "n-0",
            "n-1",
            "month_s0",
            "month_s1",
            "month_i1",
            "digest_model_i1",
            "bu_i1",
            "amt_i1",
        ],
        axis=1,
    )

    grouped = df_report.groupby(["month", "bu", "digest_model"], as_index=False)
    df_report = grouped.sum()

    df_report["amt"] = df_report["amt"].map(lambda x: x / 1000)
    df_report["amt_s1"] = df_report["amt_s1"].map(lambda x: x / 1000)
    df_report["amt_s0"] = df_report["amt_s0"].map(lambda x: x / 1000)

    df_report = df_report.reindex(
        columns=[
            "month",
            "bu",
            "digest_model",
            "qty_i1",
            "qty",
            "amt",
            "qty_s1",
            "amt_s1",
            "qty_s0",
            "amt_s0",
        ]
    )

    df_report = df_report.rename(
        columns={
            "digest_model": "model",
            "qty_i1": "last inv_q",
            "qty": "act_inv_q",
            "amt": "act_inv_kmxt",
            "qty_s1": "last_s_q",
            "amt_s1": "last_s_kmx",
            "qty_s0": "act_s_q",
            "amt_s0": "act_s_kmx",
        }
    )

    df_report = df_report.query("month == @targetMonth")
    
    try:
        df_report.to_csv("reports/admin_report_new_age.csv", index=False)
    except PermissionError as e:
        logging.exception("Raise Exception: %s", e)
    else:
        logging.info("admin_report_new_age(" + targetMonth + "): Successfully finished.")
        return df_report
    

logging.debug("return : \n" + str(admin_report_new_age("2020/8")))

 2020-09-30 17:23:09,716 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:09,879 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:10,290 -  INFO -  read_sales() : Successfully finished.
 2020-09-30 17:23:10,332 -  INFO -  read_purchase() : Successfully finished.
 2020-09-30 17:23:10,342 -  INFO -  latest_ps() : Successfully finished.
 2020-09-30 17:23:11,331 -  INFO -  read_aging() : Successfully finished.
 2020-09-30 17:23:11,340 -  INFO -  new_age() : Successfully finished.
 2020-09-30 17:23:11,374 -  INFO -  read_bu() : Successfully finished.
 2020-09-30 17:23:11,544 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:11,722 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:12,112 -  INFO -  read_sales() : Successfully finished.
 2020-09-30 17:23:12,157 -  INFO -  read_purchase() : Successfully finished.
 2020-09-30 17:23:12,167 -  INFO -  latest_ps() : Successfully finished.
 2020-09-30 17:23:13,110 -  INFO -  read_aging() : Succe

In [17]:
main()

 2020-09-30 17:23:13,907 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:13,916 -  INFO -  read_bu() : Successfully finished.
 2020-09-30 17:23:13,989 -  INFO -  bs_report_act_age(2020/3, 2020/8): Successfully finished.
 2020-09-30 17:23:14,170 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:14,444 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:14,879 -  INFO -  read_sales() : Successfully finished.
 2020-09-30 17:23:14,919 -  INFO -  read_purchase() : Successfully finished.
 2020-09-30 17:23:14,930 -  INFO -  latest_ps() : Successfully finished.
 2020-09-30 17:23:15,918 -  INFO -  read_aging() : Successfully finished.
 2020-09-30 17:23:15,928 -  INFO -  new_age() : Successfully finished.
 2020-09-30 17:23:15,950 -  INFO -  read_bu() : Successfully finished.
 2020-09-30 17:23:16,009 -  INFO -  bs_report_new_age(2020/3, 2020/8): Successfully finished.
 2020-09-30 17:23:16,169 -  INFO -  read_inv : Successfully finished.
 2020-09-30 17:23:16,